# TP d'application de Numpy : traitement d'images

Le but de ce TP est de se familiariser à numpy en appliquant les outils numpy au traitement d'image.
A cette fin, on va utiliser des outils de visualisation, qu'on consiérera simplement comme des boîtes noires permettant de passer dans un sens ou dans un autre d'une image à un tableau numpy

In [ ]:
from matplotlib.pyplot import imread, imshow
import numpy as np
from numpy.random import random

## Comprendre ce qu'est une image

Dans un premier temps quelques expérimentations pour comprendre comment une image est est représentée par un tableau numpy.

In [ ]:
petit_damier = np.array([[0,1],[1,0]])
imshow(petit_damier)

In [ ]:
imshow(petit_damier,cmap="binary_r")

**Exercice** : faire afficher un damier 8x8

**Exercice :** Afficher un damier $8\times 8$ dont les cases sont coloriées de façon aléatoire.

Nous allons maintenant télécharger une photo bien connue de tous ceux qui font du traitement d'images. Pour ceux qui voudraient en savoir plus au sujet de cette image : https://fr.wikipedia.org/wiki/Lenna

In [ ]:
lena = imread("http://optipng.sourceforge.net/pngtech/img/lena.png")

In [ ]:
imshow(lena)

Regardons comment est constitué l'objet python qui contient l'information de  l'image :

In [ ]:
print(lena.shape, lena.dtype)
print(lena)

Il est temps de jeter un coup d'oeil sur la documentation des fonction `imread` et `imshow`

In [ ]:
help(imread)

In [ ]:
help(imshow)

**Représentation des images**

Une image est une matrice dont les points sont appelés des pixels. Ainsi notre image test est une matrice de $512\times 512$ pixels.
- Si l'image est en niveaux de gris, chaque pixel est décrit par une intensité (suivant les codages, un nombre réel entre 0 et 1, ou un entier entre 0 et 255, ou ...)
- Si l'image est en couleurs, chaque pixel est décrit par un triplet d'intensités (de rouge, vert, bleu dans le codage RVB).

## Les composantes monocolores d'une image

### **Exercice** 

Extraire les composantes rouge, vert et bleu de lena (chacune d'elles est un tableau 512x512).

Pour obtenir une image en niveaux de gris, il est habituel de prendre une moyenne pondérée de ces trois composantes. Les coefficients sont choisis pour tenir compte au mieux de la sensibilité de l'oeil humain aux différentes couleurs :

``` gris = 0.299*rouge + 0.587*vert +0.114*bleu ```

A partir de la photo en couleur `lena`, fabriquez et affichez l'image en niveaux de gris correspondante, et faites-la afficher (pour un bon rendu, il faut choisir une colormap adéquate ; je conseille de mettre l'option `cmap="binary_r"` )

Si on veut afficher la composante rouge de lena, il nous faut cette fois-ci construire un nouveau tableau 512x512x3, dont la composante rouge est celle de lena, et les deux autres sont nulles.

In [ ]:
lena_R = np.zeros_like(lena)
lena_R[:,:,0] = lena_r
imshow(lena_R)

**Exercice :** faire de même avec les autres composantes de couleur de l'image.

**Exercice :** Négatif d'une image en niveaux de gris.
Créer le négatif de la photo de Lena en niveaux de gris  (un pixel d'intensité $x$ est transformé en un pixel d'intensité $1-x$. Faites-le afficher en utlisant la colormap `binary_r`.

Remarque : on aurait pu obtenir le même affichage en affichant directement `lena_gris` avec la colormap `binary`, qui est la colormap inverse de `binary_r`.

In [ ]:
imshow(lena_gris, cmap="binary")

## Luminosité et contraste

Pour aumenter la luminosité d'une image en niveaux de gris, la solution la plus simple est de décaller tous les pixels d'un même coefficient $c$. Autrement dit, on applique au tableau une fonction $f_c:x\mapsto x+c$, avec $c\in [0,1]$. 

Le problème est que pour certains pixels, la valeur $x+c$ sera supérieure à 1. On doit donc modifier cette fonction en prenant :
$f_c:x\mapsto \min(x+c,1)$.

**Exercice :** Ecrire une fonction `augmente_luminosite(M,c)` qui effectue cette opération (on pourra utiliser la fonction `np.minimum`).

Tester cette fonction en l'appliquant à  lena, avec différentes valeurs de $c$.

Le défaut de cette méthode est que les noirs profonds disparaissent et que les zones très éclairées se retrouvent saturées. Pour remédier à ceci, on peut envisager d'utiliser une fonction $f:[0,1]\to[0,1]$ croissante bijective avec $f'(0)>1$ et $f'(1)=0$. Par exemple, on peut prendre $f(x) = x(2-x)$.

**Exercice :** Ecrire une fonction `augmente_luminosite_2(M)` qui effectue cette opération.

Tester cette fonction en l'appliquant à lena.

On peut aussi envisager d'augmenter le contraste de l'image. Pour cela, il s'agit de dilater l'intervalle des valeurs des pixels autour de 1/2 par un facteur multiplicatif $c$. Autement dit, on applique à l'image une fonction 
$f_c : x\mapsto 1/2 + c(x-1/2)$. ($c$ est un réel supérieur à 1).

**Exercice :** Ecrire une fonction `augmente_contraste(M,c)` qui effectue cette opération.

Testez votre fonction avec différentes valeurs de $c$.

In [ ]:
M = lena_gris.copy()
imshow(augmente_contraste(M,1.4),cmap="binary_r")

## Détection de contours

Les contours d'une image sont les endroits où la variation d'intensité est la plus importante. Une façon de mesurer la variation d'intensité dans une image M de  taille $n\times n$ est de construire une nouvelle matrice N de taille $(n-1)\times(n-1)$, dont les coefficients sont 
$$
N_{i,j} =
\sqrt{  (M_{i,j+1}-M{i,j} )^2 + (M_{i+1,j}-M{i,j} )^2 }
$$

**Exercice** Faire afficher les contours du portrait de Lena.

**Exercice :** Maintenant que vous avez réussi à appliquer cette transformation à la photo proposée, écrivez une fonction `contour(adresse)` qui prend en argument l'adresse d'une image  monocolore, et affiche ses contours. Appliquez-la à l'image de votre choix, trouvée sur le web.

## Modification de la résolution et pixélisation

A partir d'une image $2n \times 2m$, on peut fabriquer une image $n\times m$ : on remplace chaque carré $2\times2$ de l'image initiale par un unique pixel dont la valeur  est la moyenne des 4 pixels du petit carré de l'image initiale. En faisant ça, on obtient une image de poids plus petit que l'image initiale, mais aussi on perd de l'information.

**Exercice :** Ecrire une fonction `reduction(M)`, prenant en argument une image en couleur, et rendant l'image obtenue par le procédé de réduction décrit plus haut.

Appliquer plusieurs fois successivement cette opération à notre image `lena` :

In [ ]:
N = reduction(lena)
imshow(N)

In [ ]:
N = reduction(N)
imshow(N)

In [ ]:
N = reduction(N)
imshow(N)

In [ ]:
N = reduction(N)
imshow(N)

On voit bien apparaitre le phénomène de pixélisation, qui apparaît quand on diminue trop fortement la résolution d'une image

## Débruitage d'une image

On va partir de l'image en niveau de gris, et lui appliquer un bruitage, en modifiant des pixels choisis aléatoirement.
Le but de l'exercice sera de retouver une image décente à partir de l'image bruitée.

On commence par écrire une fonction `bruitage(M,nb_pixels)` qui prend en argument une image monochrome et remplace la valeur de `nb_pixels` choisis aléatoirement par 1  (pixel blanc) :

In [ ]:
from numpy.random import randint
def bruitage(M,nb_pixels):
    N = M.copy()
    l,m = N.shape
    x = randint(0,l,nb_pixels)
    y = randint(0,m,nb_pixels)
    N[x,y] = 1
    return N

In [ ]:
orig = lena_gris.copy()
image_bruitee = bruitage(orig,2000)
imshow(image_bruitee,cmap="binary_r")

**Exercice :** Votre mission est maintenant de rectifier cette image. Pour cela, écrivez une fonction `filtre(M,k)` prenant en argument une image monochrome et un entier $k$, et et rendant l'image obtenue en remplaçant chaque pixel par la moyenne des pixels du carré de taille $2k+1$ centré autour de ce pixel.

Appliquez votre filtre à l'image bruitée, avec $k=1$, et faites afficher l'image.
Que donne ce filtre avec des valeurs de $k$ plus grandes ?

**Exercice :** Écrivez un filtre analogue `filtre_mediane(M,k)`, où on prend cette fois-ci la moyenne des $k^2$ pixels voisins d'un pixel.

Testez ce nouveau filtre sur l'image bruitée.

## La transformation du photomaton

On va transformer notre image couleur de taille 512x512 de base en construisant 4 images de taille 156x256 de la façon suivante

- $M_{1,1}$ contient tous les pixels d'abscisse paire et d'ordonnée paire
- $M_{2,1}$ contient tous les pixels d'abscisse impaire et d'ordonnée paire
- $M_{1,1}$ contient tous les pixels d'abscisse paire et d'ordonnée impaire
- $M_{2,1}$ contient tous les pixels d'abscisse impaire et d'ordonnée impaire

et on construit une nouvelle matrice $N$ en juxtaposant ces 4 matrices

$$
N =\left( \begin{array}{cc} 
M_{1,1} & M_{1,2}\\
M_{2,1} & M_{2,2}
\end{array} \right)
$$

**Exercice :**  Construire une fonction `photomaton(M)` prenant en argument le tableau M et rendant le tableau N.

**Question :** Que se passe-t-il si on itère 9 fois cette transformation à partir de la photo de Lena ?
Savez-vous interpréter mathématiquement le phénomène que vous observez ?

In [ ]:
M = lena.copy()
imshow(M)

In [ ]:
M = photomaton(M)# première itération de la transformation
imshow(M)  

In [ ]:
M = photomaton(M)# deuxième itération de la transformation
imshow(M)  

In [ ]:
M = photomaton(M)  # troisième itération
imshow(M)

In [ ]:
M = photomaton(M)  # septième itération
imshow(M)

In [ ]:
M = photomaton(M)  # quatrtième itération
imshow(M)

In [ ]:
M = photomaton(M)  # cinquième itération
imshow(M)

In [ ]:
M = photomaton(M)  # sixième itération
imshow(M)

In [ ]:
M = photomaton(M)  # septième itération
imshow(M)

In [ ]:
M = photomaton(M)  # huitème itération
imshow(M)

In [ ]:
M = photomaton(M)  # et voilà la neuvième itération
imshow(M)

On voit qu'à la neuvième itération on récupère l'image initiale. La transformation du photomaton réalise une permutation $\sigma$ de l'ensemble des $512\times512$ pixels de l'image. La question est alors de prouver mathématiquement que $\sigma^9 = \text{Id}$.